In [ ]:
import os
import sys 
import typing as t
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
from skimage.feature import greycomatrix
from skimage.color import rgb2hsv, rgb2ycbcr

# add src to path
sys.path.append('../..')
from utils.db_helper import get_image_data
# from utils.color_space_operations import all_colorspaces_from_rgb

In [ ]:
SOURCE_DIR_STYLE = R"F:\master-thesis-databases\classification_db\fake"
SOURCE_DIR_FFHQ = R"F:\master-thesis-databases\classification_db\real"

In [ ]:
GRAYSCALE=False
base_save_path = os.path.join("..", "..", "..", "exp", "Comatrix")
DISTANCES = [1,2]
ANGLES = [0, np.pi/4, np.pi/2, 3*np.pi/4]

In [ ]:
def comatrix_from_image(np_img: np.ndarray, distances: t.List[int], angles: t.List[float])-> np.ndarray:
    if len(np_img.shape) == 3:
         np_comatrix = np.empty((256,256,np_img.shape[-1],len(distances),len(angles)))
        for i in range(np_img.shape[-1]):
            np_comatrix[:,:,i] = greycomatrix(np_img[:,:,i], distances, angles)
    else:
        raise ValueError('Image array must have 3 dimensions')
    return np_comatrix

In [ ]:
def comatrix_from_dataset(src_path: str, distances: t.List[int], angles: t.List[float], grayscale=False)-> np.ndarray:
    print("Loading dataset...")
    dataset_gen = get_image_data(src_path, type='int', grayscale=grayscale)
    final_type = np.float64
    print("Processing database...")
    np_comatrix_dataset = np.zeros((256,256,9,len(distances),len(angles)), 
                                   dtype=final_type)
    number_of_images = 0
    for np_image in dataset_gen:
        np_image = all_colorspaces_from_rgb(np_image, type='int')
        np_tmp_comatrix = comatrix_from_image(np_image, distances, angles)
        np_comatrix_dataset += np_tmp_comatrix
        number_of_images +=1
        if number_of_images % 100 == 0:
            print(f"Image number: {number_of_images}")
        elif np_comatrix_dataset.max() > 0.95*np.finfo(final_type).max:
            raise ValueError("Dataset to large, datatype overflow")
            break
    print (f"{number_of_images} processed from path: {src_path}")
    return np_comatrix_dataset / number_of_images

In [ ]:
# def display_comatrix(np_comatrix_dataset: np.ndarray):
np_comatrix_ffhq = comatrix_from_dataset(SOURCE_DIR_FFHQ, DISTANCES, ANGLES, grayscale=False)
np_comatrix_style = comatrix_from_dataset(SOURCE_DIR_STYLE, DISTANCES, ANGLES, grayscale=GRAYSCALE)

In [ ]:
datasets = {
    'real': np_comatrix_ffhq,
    'fake': np_comatrix_style
}

color_componenets = ("R", "G", "B", "H", "S", "V", "Y", "Cb", "Cr")

if GRAYSCALE:
    color_channels = {
    "0": 'Gray',
    }
else:
    color_channels = {
        "0": 'R',
        "1": 'G',
        '2': 'B',
        '3': 'H',
        '4': 'S',
        '5': 'V',
        '6': 'Y',
        '7': 'Cb',
        '8': 'Cr'
    }

In [ ]:
i=0
for distance in DISTANCES:
    j=0
    for angle in ANGLES:
        fig, axs = plt.subplots(nrows=int(len(datasets)*len(color_channels)/3), ncols=3, figsize=(15,15))
        fig.suptitle(f'Comatrixes for distance: {distance} and angle: {angle} rads')
        plt.subplots_adjust(hspace=0.5)
        row=0
        dataset_nr=0
        for name, dataset in datasets.items():
            # if len(color_channels) > 1:
            for col in range(len(color_channels)):
                if col%3 == 0 and col>0:
                    row+=1
                axs[row+dataset_nr,col%3].imshow(dataset[:,:,col,i,j])
                axs[row+dataset_nr,col%3].set_title(f"{name}_{color_channels[str(col)]}")
            dataset_nr+=1
            # else:
            #         axs[row].imshow(dataset[:,:,i,j])
            #         axs[row].set_title(f"{name}")
            #row+=1
        #saving image
        #directory = 'Grayscale' if GRAYSCALE else 'Colors'
        filename = f"{distance}_{angle}.png"
        save_path = os.path.join(base_save_path, filename)
        #plt.show()
        fig.savefig(save_path)
        j+=1
    i+=1